MATLBA R2018a
```matlab
function [out] = test(n, m)

% generate random mean (mu) and covariance matrix (sigma)
out = struct;
out.mu = randn(n, 1);
q = qr(randn(n, n));
out.sigma = q * q';

% compute intermediate values for a, b, and c
std = realsqrt(diag(out.sigma));
std_outer = std' .* std;
sigma_det = max(std_outer - out.sigma, 0) .* (std_outer + out.sigma);
sqrt_sigma_det = realsqrt(sigma_det);
mean_std = sqrt(1 / 2) .* out.mu ./ std;
irho = std_outer ./ sqrt_sigma_det;

% compute a, b, and c
out.a = irho .* out.sigma ./ std_outer;
out.b = irho .* mean_std;
out.c = repmat(-mean_std', n, 1);
out.b(irho == Inf) = 0;

% save the results of every number of terms from 0 to m - 1
out.r = zeros(m, n, n);
for t = 0:double(m) - 1
    out.r(t + 1, :, :) = erf_exp_integral(out.a, out.b, out.c, t);
end

% save best integral value
out.t = erf_exp_integral(out.a, out.b, out.c);

end

function [y] = erf_exp_integral(a, b, c, n)
%ERF_EXP_INTEGRAL Integrate erf(a.*x+b)/exp(x.^2) over x from c to Inf
%   If n >= 0, use the n terms from the infinite sum of the integral.
%   Otherwise, use the best numerical integration method (preferred).

if nargin < 4 || n < 0
    f = @(a, b, c) integral(@(x) erf(a .* x + b) ./ exp(x.^2), c, Inf);
    y = arrayfun(f, a, b, c);
else  % this is implemented for educational purposes only
    % if abs(a) < 1
    y = I(a, b, Inf, n) - I(a, b, c, n);

    % else
    idx = abs(a) >= 1;
    a = a(idx); b = b(idx); c = c(idx);
    upper = I(1./abs(a), -b./a, Inf, n);
    lower = I(1./abs(a), -b./a, (a.*c+b).*sign(a), n);
    y(idx) = (-pi/4).*erf(a.*c+b).*erf(c) + sign(a).*((pi/4)+lower-upper);

    y = y ./ sqrt(pi/4);
end
end

function [y] = I(a, b, x, n)
term1 = (pi / 4) .* erf(x) .* erf(b);
term2 = sqrt(pi / 4) .* exp(-b.^2) .* series(a, b, x, n);
y = term1 + term2;
end

function [y] = series(a, b, x, n)
% syms u
% y = double(symsum(gammaH(u+1,x,a,b)-sign(x).*gammaH(u+1.5,x,a,b),u,0,n));
y = 0;
for u=0:n
    y = y + gammaH(u + 1, x, a, b) - sign(x).*gammaH(u + 1.5, x, a, b);
end
end

function [y] = gammaH(nu, z, a, b)
y = hermiteH(2 .* (nu - 1), b) .* gammainc(nu, z.^2) ./ gamma(nu + 0.5);
y = (a ./ 2) .^ (2 .* (nu - 0.5)) .* y;
end

function [y] = gammainc(nu, z)
y = 1 - igamma(nu, z) ./ gamma(nu);
end
```

In [0]:
import torch
import numpy as np
from matlab.engine import start_matlab
# https://www.mathworks.com/help/matlab/matlab-engine-for-python.html


M = start_matlab()
to_array = lambda a: np.asarray(a._data).reshape(a.size[::-1]).T
to_tensor = lambda a: torch.from_numpy(to_array(a))
M.path('/home/modar/Dropbox/gaussian_relu_moments', M.path())
o = {k: to_tensor(v) for k, v in M.test(3, 10).items()}

Python 3.7.6, PyTorch 1.4.0, SciPy 1.3.2

In [0]:
from math import gamma, pi, sqrt

import numpy as np

import scipy
import scipy.integrate
import scipy.special

import torch
from torch.autograd import Function


def non_differentiable(function):
    name = function.__qualname__

    @staticmethod
    def forward(ctx, *args, **kwargs):  # pylint: disable=unused-argument
        with torch.no_grad():
            return function(*args, **kwargs)

    return type(name, (Function,), {'forward': forward}).apply


@np.vectorize
def numpy_erf_exp_integral(a, b, c):
    """Integrate `erf(a*x+b) * exp(-x**2)` from `c` to infinity in NumPy."""
    f = lambda x: scipy.special.erf(a * x + b) * np.exp(-x * x)
    return scipy.integrate.quad(f, c, np.inf)[0]


@non_differentiable
def torch_erf_exp_integral(a, b, c):
    """Integrate `erf(a*x+b) * exp(-x**2)` from `c` to infinity in PyTorch."""
    a_ = a
    a = a.detach().cpu().numpy()
    b = b.detach().cpu().numpy()
    c = c.detach().cpu().numpy()
    return a_.new(numpy_erf_exp_integral(a, b, c))


def polynomial(coefficients, x):
    out = 0
    coefficients = iter(coefficients)
    out += next(coefficients, 0)
    for c in coefficients:
        out *= x
        out += c
    return out


def hermite(n, x):
    return polynomial(scipy.special.hermite(n).coeffs.tolist(), x)


@non_differentiable
def gammainc(nu, z):
    # torch.distributions.Gamma().cdf(v) is not implemented yet
    return z.new(scipy.special.gammainc(nu, z.detach().cpu().numpy()))


def series(a, b, x, n):
    a, x, s = a / 2, x * x, -x.sign()

    def term(nu):
        gammas = gammainc(nu, x) / gamma(nu + 0.5)
        return a**(2 * (nu - 0.5)) * hermite(2 * (nu - 1), b) * gammas

    out = 0
    for i in range(n + 1):
        out += term(i + 1) + s * term(i + 1.5)
    return out


def definite_differece(a, b, x, n):
    inf = x.new([float('inf')]).expand(x.size())
    s = series(a, b, inf, n) - series(a, b, x, n)
    return sqrt(pi / 4) * b.erf() * (1 - x.erf()) + (-b * b).exp() * s


def erf_exp_integral(a, b, c, n=-1):
    if n < 0:
        return torch_erf_exp_integral(a, b, c)
    y = definite_differece(a, b, c, n)
    idx = a.abs() >= 1
    a, b, c = a[idx], b[idx], c[idx]
    acb = a * c + b
    v = definite_differece(1 / a.abs(), -b / a, acb * a.sign(), n)
    v = a.sign() * (sqrt(pi / 4) - v) - sqrt(pi / 4) * acb.erf() * c.erf()
    return y.masked_scatter_(idx, v)

In [0]:
# compare term sums
atol, rtol = 1e-8, 1e-2
for i, r in enumerate(o['r']):
    s = erf_exp_integral(o['a'], o['b'], o['c'], i)
    # print(((s - r).abs() - rtol * rz.abs() - atol).clamp(0))
    print('Error', (s - r).norm().item())

# compare integration method
t = erf_exp_integral(o['a'], o['b'], o['c'])
print('Error', (t - o['t']).norm().item())
print('Error', (o['r'][-1] - o['t']).norm().item())